<a href="https://colab.research.google.com/github/gundumeghana/AI-Assisted-Coding/blob/main/2403A510C1_17_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import re

# --- Setup and Task 1 ---
df = pd.read_csv('customerfeedback.csv')

# 1. Fill missing 'rating' values with the median
median_rating = df['rating'].median()
df['rating'].fillna(median_rating, inplace=True)

# 2. Standardize 'date' column to YYYY-MM-DD format
df['date'] = pd.to_datetime(
    df['date'],
    errors='coerce', # Set unparseable dates to NaT (which becomes 'nan' after strftime)
    infer_datetime_format=True
).dt.strftime('%Y-%m-%d')


# --- Task 2: Text Cleaning ---

# Define a simple spelling correction dictionary for common typos
spell_check_dict = {
    'amazng': 'amazing',
    'packging': 'packaging',
    'servce': 'service',
    'experiance': 'experience',
    'prodct': 'product'
}

# Hardcoded list of common English stopwords (to ensure execution without NLTK download)
stop_words = set(['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"])

def clean_text(text):
    # a. Convert text to lowercase
    text = text.lower()

    # b. Correct common spelling mistakes (simplified)
    words = text.split()
    corrected_words = [spell_check_dict.get(re.sub(r'[^a-z]', '', word), word) for word in words]
    text = ' '.join(corrected_words)

    # c. Remove punctuation and split into words
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()

    # d. Remove stopwords and single-character tokens
    filtered_words = [word for word in words if word not in stop_words and len(word) > 1]

    return ' '.join(filtered_words)

# Apply the cleaning function
df['cleaned_feedback'] = df['feedback_text'].apply(clean_text)

# Final step: Save the processed data
df.to_csv('customerfeedback_fully_processed.csv', index=False)

/tmp/ipython-input-3780803915.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['rating'].fillna(median_rating, inplace=True)
/tmp/ipython-input-3780803915.py:12: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df['date'] = pd.to_datetime(


In [8]:
import pandas as pd
import numpy as np

# Load the Medical Records Dataset
# NOTE: Ensure 'Medical_Records_Dataset.csv' is uploaded and accessible.
try:
    df_medical = pd.read_csv('Medical_Records_Dataset.csv')
except FileNotFoundError:
    print("Error: 'Medical_Records_Dataset.csv' not found. Please upload the file.")
    exit()

print("Initial data loaded. Starting preprocessing for Medical Records...")

# ==============================================================================
#                      TASK 1: Detect and Handle Outliers in 'blood_pressure'
# ==============================================================================
column_name = 'blood_pressure'

# 1. Calculate IQR and bounds
Q1 = df_medical[column_name].quantile(0.25)
Q3 = df_medical[column_name].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 2. Cap/Winsorize outliers
# Replace values outside bounds with the respective bounds
outliers_count = df_medical[
    (df_medical[column_name] < lower_bound) | (df_medical[column_name] > upper_bound)
].shape[0]

df_medical[column_name] = np.where(
    df_medical[column_name] < lower_bound,
    lower_bound,
    df_medical[column_name]
)
df_medical[column_name] = np.where(
    df_medical[column_name] > upper_bound,
    upper_bound,
    df_medical[column_name]
)

print(f"Task 1 Complete: Outliers in '{column_name}' capped (Total detected: {outliers_count}).")


# ==============================================================================
#                      TASK 2: Standardize and Encode 'gender'
# ==============================================================================

# 1. Standardization function to handle inconsistent labels
def standardize_gender(gender):
    if pd.isna(gender):
        return 'Unknown'
    gender = str(gender).lower().strip()
    if gender in ['male', 'm', 'man', 'boy']:
        return 'Male'
    elif gender in ['female', 'f', 'woman', 'girl']:
        return 'Female'
    else:
        # Catch any remaining inconsistent values
        return 'Other'

df_medical['gender_standardized'] = df_medical['gender'].apply(standardize_gender)

# 2. One-Hot Encoding
# drop_first=True is used to avoid multicollinearity by dropping the first category ('Female' if sorted, or 'Male' if the default order is used)
df_gender_encoded = pd.get_dummies(
    df_medical['gender_standardized'],
    prefix='gender',
    drop_first=True
)

# Merge the encoded columns and drop the original and intermediary columns
df_medical = pd.concat(
    [
        df_medical.drop(['gender', 'gender_standardized'], axis=1),
        df_gender_encoded
    ],
    axis=1
)

print("Task 2 Complete: Gender standardized and One-Hot Encoded.")

# --- Final Step: Save the processed data to a new CSV file ---
output_filename = 'medicalrecords_fully_processed.csv'
df_medical.to_csv(output_filename, index=False)

print(f"\n--- Preprocessing Complete ---")
print(f"Processed data saved to: {output_filename}")
print("Final Data Head (showing processed 'blood_pressure' and encoded 'gender'):")
print(df_medical.head().to_markdown(index=False))

Initial data loaded. Starting preprocessing for Medical Records...
Task 1 Complete: Outliers in 'blood_pressure' capped (Total detected: 1).
Task 2 Complete: Gender standardized and One-Hot Encoded.

--- Preprocessing Complete ---
Processed data saved to: medicalrecords_fully_processed.csv
Final Data Head (showing processed 'blood_pressure' and encoded 'gender'):
| patient_id   |   age |   blood_pressure |   cholesterol | gender_Male   |
|:-------------|------:|-----------------:|--------------:|:--------------|
| P001         |    25 |              120 |           180 | True          |
| P002         |    45 |              180 |           250 | False         |
| P003         |    36 |              130 |           200 | True          |
| P004         |    29 |              110 |           170 | False         |
| P005         |    67 |              215 |           300 | True          |


In [7]:
import pandas as pd
import numpy as np

# --- Setup: Load the Financial Transactions Dataset ---
# NOTE: This script assumes a file named 'Financial_Transactions_Dataset.csv' is available.
try:
    df_finance = pd.read_csv('Financial_Transactions_Dataset.csv')
except FileNotFoundError:
    print("Error: 'Financial_Transactions_Dataset.csv' not found. Please upload the file.")
    exit()

print("Initial data loaded. Starting preprocessing for Financial Transactions...")

# ==============================================================================
#                      TASK 1: Remove Duplicates and Convert Currency
# ==============================================================================

# 1. Remove duplicate transactions (keeping the first occurrence)
initial_rows = df_finance.shape[0]
df_finance.drop_duplicates(inplace=True)
duplicates_removed = initial_rows - df_finance.shape[0]
print(f"Removed {duplicates_removed} duplicate transactions.")

# 2. Define a currency conversion dictionary (example rates relative to USD)
CONVERSION_RATES = {
    'USD': 1.0,
    'EUR': 1.08,  # Example: 1 EUR = 1.08 USD
    'INR': 0.012, # Example: 1 INR = 0.012 USD
    'GBP': 1.25   # Example: 1 GBP = 1.25 USD
}

# 3. Conversion function to convert 'amount' to 'amount_USD'
def convert_to_usd(row):
    currency = str(row['currency']).upper().strip()
    amount = row['amount']
    # Use .get() with a default value (1.0) for safety
    rate = CONVERSION_RATES.get(currency, 1.0)
    return amount * rate

# Apply the conversion function to create the new column
df_finance['amount_USD'] = df_finance.apply(convert_to_usd, axis=1)
print("Amounts converted to USD using the provided dictionary.")

# ==============================================================================
#                      TASK 2: Normalize Timestamp and Extract Hour
# ==============================================================================

# 1. Normalize the 'timestamp' column to UTC
df_finance['timestamp_utc'] = pd.to_datetime(
    df_finance['timestamp'],
    errors='coerce', # Handle unparseable dates by converting them to NaT
    utc=True         # Normalize all timestamps to UTC timezone
)

# 2. Create 'transaction_hour' column (Hour of the day in UTC time, 0-23)
df_finance['transaction_hour'] = df_finance['timestamp_utc'].dt.hour
print("Timestamp normalized to UTC and 'transaction_hour' extracted.")

# 3. Drop the original 'timestamp' and 'currency' columns for a cleaner final dataset
df_finance.drop(columns=['timestamp', 'currency'], inplace=True, errors='ignore')

# --- Final Step: Save the processed data to a new CSV file ---
output_filename = 'financialtransactions_fully_processed.csv'
df_finance.to_csv(output_filename, index=False)

print(f"\n--- Preprocessing Complete ---")
print(f"Processed data saved to: {output_filename}")
print("Final Data Head (showing new columns):")
print(df_finance.head().to_markdown(index=False))

Initial data loaded. Starting preprocessing for Financial Transactions...
Removed 0 duplicate transactions.
Amounts converted to USD using the provided dictionary.
Timestamp normalized to UTC and 'transaction_hour' extracted.

--- Preprocessing Complete ---
Processed data saved to: financialtransactions_fully_processed.csv
Final Data Head (showing new columns):
| transaction_id   |   amount | merchant   |   amount_USD | timestamp_utc             |   transaction_hour |
|:-----------------|---------:|:-----------|-------------:|:--------------------------|-------------------:|
| T001             |     1000 | Amazon     |         1000 | 2025-10-01 10:00:00+00:00 |                 10 |
| T002             |     1500 | Flipkart   |           18 | NaT                       |                nan |
| T003             |     2000 | eBay       |         2160 | NaT                       |                nan |
| T004             |     1000 | Amazon     |         1000 | NaT                       |    

/tmp/ipython-input-3103997052.py:49: FutureWarning: Parsed string "2025-10-01 10:00:00 PST" included an un-recognized timezone "PST". Dropping unrecognized timezones is deprecated; in a future version this will raise. Instead pass the string without the timezone, then use .tz_localize to convert to a recognized timezone.
  df_finance['timestamp_utc'] = pd.to_datetime(
